In [1]:
module cse

using DataStructures: OrderedDict

immutable CacheElement
    name::Symbol
    expr::Nullable{Expr}
end

typealias Cache OrderedDict{Symbol, CacheElement}

cacheify!(setup, expr) = expr

function cacheify!(setup::Cache, expr::Expr)
    if expr.head == :call
        available = all([any([arg == s.name for s in values(setup)]) for arg in expr.args])
        if available
            cached_name = Symbol(expr.args...)
            if !haskey(setup, cached_name)
                cached_varname = gensym(cached_name)
                    setup[cached_name] = CacheElement(cached_varname, :($cached_varname = $(copy(expr))))
            else
                cached_varname = setup[cached_name].name
            end
            return cached_varname
        end
    end
          
    for (i, child) in enumerate(expr.args)
        expr.args[i] = cacheify!(setup, child)
    end
    expr
end
    
    disqualify_symbols!(available, disqualified, expr) = nothing
    disqualify_symbols!(available, disqualified, expr::Symbol) = push!(disqualified, expr)
    
    
    function disqualify_symbols!(available, disqualified, expr::Expr)
        for arg in expr.args
            disqualify_symbols!(available, disqualified, arg)
        end
    end
    
    available_leaf_symbols!(available, disqualified, expr) = nothing
    available_leaf_symbols!(available, disqualified, expr::Symbol) = push!(available, expr)

    function available_leaf_symbols!(available, disqualified, expr::Expr)
        if expr.head == :line
            # nothing
        elseif expr.head == :(=)
            disqualify_symbols!(available, disqualified, expr.args[1])
            for arg in expr.args[2:end]
                available_leaf_symbols!(available, disqualified, arg)
            end
        else
            for arg in expr.args
                available_leaf_symbols!(available, disqualified, arg)
            end
        end
    end
        
    function available_leaf_symbols(expr)
        available = Set{Symbol}()
        disqualified = Set{Symbol}()
        available_leaf_symbols!(available, disqualified, expr)
        setdiff!(available, disqualified)
    end
    
function cacheify(expr::Expr)
    setup = Cache()
        for var in available_leaf_symbols(expr)
            setup[var] = CacheElement(var, nothing)
        end
    while true
        num_setup = length(setup)
        expr = copy(expr)
        expr = cacheify!(setup, expr)
        if length(setup) == num_setup
            break
        end
    end
        Expr(:block, [get(v.expr) for v in values(setup) if !isnull(v.expr)]..., expr)
end

    macro cse(expr)
        result = cacheify(expr)
        println(result)
        esc(result)
    end
    

    
end

cse

In [6]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

h(x, y) = 3

x = 1

expr = quote
    [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
end

quote  # In[6], line 16:
    [h(h(f(g(x)),g(x)),i) == i for i = 1:5]
end

In [7]:
cse.cacheify(expr)

quote 
    ##gx#276 = g(x)
    ##f##gx#276#277 = f(##gx#276)
    ##h##f##gx#276#277##gx#276#278 = h(##f##gx#276#277,##gx#276)
    begin  # In[6], line 16:
        [h(##h##f##gx#276#277##gx#276#278,i) == i for i = 1:5]
    end
end

In [8]:
@cse.cse begin
    [f(f(f(g(x)), g(x)), i) == i for i in 1:5]
end

begin 
    ##gx#279 = g(x)
    ##f##gx#279#280 = f(##gx#279)
    ##f##f##gx#279#280##gx#279#281 = f(##f##gx#279#280,##gx#279)
    begin  # In[8], line 2:
        [f(##f##f##gx#279#280##gx#279#281,i) == i for i = 1:5]
    end
end
g
f


5-element Array{Bool,1}:
 false
 false
  true
 false
 false